# Movie Search API - Project

### Introduction to the project

Welcome to my project! The purpose of this project is to effectively design a Python program with the main functionality to allow users to search for information about movies contained in the Open Movie Database (omdb). This file starts with Section A, containing the full functional program code, and then is followed by Section B outlining the documentation explaining some core concepts, the code, retrospective, and help documentation.

#### Core Functionality
My program has two main functionalities:
1. Allowing users to search for new movies in the movies database
2. Allowing users to view information about movies that they have previously searched. 

If they search for a new movie, then they first given the default response of a movie match with the most recent year or most accurate match. They are then given the option to specify their search through providing a year (with the options of the years and movie titles being provided). If they wish to get information about a movie that they have previously searched for, then they should provide the exact title of that movie (with the list of titles being displayed for them to choose from). 



### Help documentation
1. Install and import all necessary modules. This is done automatically by running block 1. If an error persists stating that you do not have a module, then just add that to your installed modules through `!pip install <moduleName>` in the first block.
2. Run the program by running the second block. Enter 1 to search for information of a movie that you have already searched for. Enter 2 to search for a new movie. Follow the instructions presented.
3. You can quit the program at any point by entering 'q' when asked for user input.
4. You can be directed to the main menu at any point by entering 'm' when asked for user input.
5. If an error occurs, it is likely to be that the connection timed out. To solve this problem, try rerunning the first block (connecting the api key), or restart the kernel and make sure to run 1) and 2) again.
6. Navigate to Section B for more documentation.

## Section A: Implementation

I have decided to display my whole program in a single code cell and have used the python Markdown feature throughout my coding to provide explanations for my program design decisions.

In [1]:
#installing omdb module with pip
! pip install omdb 

#importing python built in modules so I will have access to the functions defined in the respective modules.
import omdb 
import json
import sys
from pathlib import Path
!{sys.executable} -m pip install --upgrade pip

# setting the default API key
omdb.set_default('apikey', "48598600") 

In [3]:
# format text
bold = "\033[1m"
red = "\033[91m"
green = "\033[92m"
blue = "\033[94m"
end_formatting = "\033[0m"



def print_user_output(title, genre, released, runtime, director, awards):
    '''
    Prints relevant fields to the user.

    @param title - movie title (String)
    @param genre - movie genre (String)
    @param released - movie realease date (String)
    @param runtime - movie runtime (String)
    @param director - movie director (String)
    @param awards - movie awards (String)
    '''
    print("{}\n{}\n{}\n{}\n{}\n{}\n".format(title, genre, released, runtime, director, awards))

def make_summary_dict(title, genre, released, runtime, director, awards):
    '''
    Creates a summary dictonary with the given fields.

    @param title - movie title (String)
    @param genre - movie genre (String)
    @param released - movie realease date (String)
    @param runtime - movie runtime (String)
    @param director - movie director (String)
    @param awards - movie awards (String)

    @return the summary dictionary (dict)
    '''
    summary_dict = {}
    summary_dict['title'] = title
    summary_dict['genre'] = genre
    summary_dict['released'] = released
    summary_dict['runtime'] = runtime
    summary_dict['director'] = director
    summary_dict['awards'] = awards
    return summary_dict
    
def initialize_file():
    '''
    Initialises the search history file if it does not exist.
    '''
    with open('search_history.txt', 'w') as file: 
        empty_array = [] 
        json.dump(empty_array, file)
        

def intro():
    '''
    Introduces the program service to the user.
    '''
    print("The purpose of this service is to allow you to search for information for a movie or tv show by the title in our database.\n")

    get_history()
    print("\n{}Enter q at any point to quit the program.\nEnter m at any point to be directed to the main menu.{}".format(blue, end_formatting))


def menu():
    '''
    Displays main menu to user and reacts according to the user's input.
    '''
    try:
        while True:
            user_search_option = input("\n{}This is the main menu.{}\nEnter 1 if you would like to view the information for one of your previously looked up movies listed above\nEnter 2 if you want to look up a new movie\nEnter q to quit the program: ".format(bold, end_formatting))
            if user_search_option == '1':
                if search_old() == 'q':
                    break
            elif user_search_option == '2':
                if search_new() == 'q':
                    break
            elif user_search_option == 'q':
                break
            else:
                continue 
        print('\n{}You have quit the program. Thank you for using our services.{}'.format(green, end_formatting))
    except Exception as e:
        print("{}Oops... seems like an error occured. Please run our program again.{}".format(red, end_formatting))
        print("{}ERROR:{}".format(bold, end_formatting))
        print(str(e))

def get_history():
    '''
    Gets the user's search history (if they have one) and displays the titles and years of all previously
    searched movies.
    '''
    # checks if the file exists, if it does not exist a new file is created 
    my_file = Path("search_history.txt")
    if not my_file.is_file():
        print("{}You have no search history, but let's work on that!{}".format(green, end_formatting))
        initialize_file()
        return
    
    # displays the movie title and years of the users previous search history, if they have one.
    search_array = []
    with open('search_history.txt', 'r') as f:
        search_array = json.loads(f.read())

    print("\n{}Here is your previous search history: {}".format(bold, end_formatting))

    for dict_history in search_array:
        print("Title: {}, Year: {}".format(dict_history['title'], dict_history['released']))

        
class Entry():
    '''
    Contains the title and year of the search result and allows for sorting by year.
    '''
    def __init__(self, title, year):
        self.title = title
        self.year = year
    def __gt__(self, other_entry):
        return self.year > other_entry.year
    def __eq__(self, other_entry):
        return self.year == other_entry.year
    def __gte__(self, other_entry):
        return self.year >= other_entry.year
    def __str__(self):
        return f'Title: {self.title}, Year: {self.year}'
    def __repr__(self):
        return str(self)

    
def read_write_search_history(response):
    '''
    Reads the search history file and writes the search response to it.
    
    @param response - the response that should be written to the search history file (JSON)
    '''
    search_array = []
    with open('search_history.txt', 'r') as f:
        search_array = json.loads(f.read())  
    if response in search_array:
        print("{}Oeee! Searching for this movie again? Must be cool!\n{}".format(green, end_formatting))
    else:
        with open('search_history.txt', 'w') as file:
            search_array.append(response)
            json.dump(search_array, file) 
            
def search_old():
    '''
    Allows for users to get information of previously searched movies by providing the title and year.
    Displays search history upon request.
    '''
    while True:
        user_title = input("\nPlease type in the movie {}title{} you would like to search for.\nEnter {}h{} to see your previous titles in your search history: \n".format(bold, end_formatting, bold, end_formatting))
        if user_title == "m" or user_title == "menu":
            break
        if user_title == "q" or user_title == "quit":
            return "q"
        if user_title == "h":
            get_history()
            continue
        
        user_year = input("\nPlease type in the movie {}year{} you would like to search for.\nEnter {}h{} to see your previous titles in your search history: \n".format(bold, end_formatting, bold, end_formatting))
        if user_year == "m" or user_year == "menu":
            break
        if user_year == "q" or user_year == "quit":
            return "q"
        if user_year == "h":
            get_history()
            continue

        # checks if the movie title and year is found in the user's search history
        search_array = []
        with open('search_history.txt', 'r') as f:
            search_array = json.loads(f.read())
        
        found_result = 0
        for dict_history in search_array:
            if user_title.lower() in dict_history['title'].lower() and user_year == dict_history['released'][-4:]:
                print_user_output(dict_history['title'], dict_history['genre'], dict_history['released'], dict_history['runtime'], dict_history['director'], dict_history['awards'])
                found_result = 1
                break
        if found_result == 0:
            print("{}Search result does not match your previous search history. Please make sure that your movie title and year exactly matches your search history.".format(red, end_formatting))
        
        

def search_new():
    '''
    Allows for users to get information of a movie in the OMDB database by providing the title and (optionally) year.
    Provides a default response initially, whereafter it allows the user to customise response by providing a year.
    Writes search result to search history file (if it is the first time the user is searching for it).
    '''
    while True:
        user_title = input("\n{}Please type in the movie title you would like to search for: {}".format(blue, end_formatting))
        if user_title == "m" or user_title == "menu":
            break
        if user_title == "q" or user_title == "quit":
            return "q"

        full_response = omdb.get(title=user_title, fullplot=True, tomatoes=True)
    
        # if title not found in omdb, then respond accordingly
        if not full_response:
            print("\n{}Your search query was not found in our database, please search again{}".format(red, end_formatting))
            continue
        print("We have found a match for {}{}{}, here is the match for the most exact match or the match with the most recent year ({}{}{})".format(bold, full_response['title'], end_formatting, bold, full_response['year'], end_formatting))
        print_user_output(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])


        # dict of appropriate fields for the user's search query 
        default_response = make_summary_dict(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])

        # search omdb to find results matching the user's search query
        search_result = omdb.search(user_title) 
        
        # displays additional matches to the movie title given (only display titles that will yield in a valid get() query)
        print("Loading more results matching your search query... ")
        search_result_entries = []
        years_list = []
        for movie_dict in search_result:
            if omdb.get(title=user_title, year=movie_dict['year']):
                entry = Entry(movie_dict['title'], movie_dict['year'])
                search_result_entries.append(entry)
                years_list.append(movie_dict['year'])
        
        print("{}Here are more results matching your search query: {}".format(bold, end_formatting))    
        # sort search results by year
        sorted_entries_results = sorted(search_result_entries, reverse=True)
        for entry in sorted_entries_results:
            print("The title is: {}, The year is: {}".format(entry.title, entry.year))
    
        
        # user can customise response by year
        user_year = input("\nIf you are interested in getting results for one of the above matches instead, please type in the associated {}year{} of that result. Otherwise, just press ENTER\n".format(bold, end_formatting))
        if user_year == "m" or user_year == "menu":
            break
        if user_year == "q" or user_year == "quit":
            return "q"
        
        if user_year in years_list:
            full_response = omdb.get(title=user_title, year=user_year, fullplot=True, tomatoes=True)
            print_user_output(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])
            specific_response = make_summary_dict(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])

            # read/write search history - write only if the user has not searched for the movie before
            read_write_search_history(specific_response)
           
        # read/write search history - write only if the user has not searched for the movie before
        else:
            print("\n{}You have not entered one of the years listed above, please search again. {}".format(red, end_formatting))
            read_write_search_history(default_response)

# run program
intro()
menu()



The purpose of this service is to allow you to search for information for a movie or tv show by the title in our database.


Here is your previous search history: 
Title: Alyson Stoner: Dancing in the Moon Light, Year: N/A
Title: Mr. Plinkett's Star Trek 2009 Review, Year: 01 Sep 2010
Title: Italian Spiderman, Year: 08 Nov 2007

Enter q at any point to quit the program.
Enter m at any point to be directed to the main menu.

This is the main menu.
Enter 1 if you would like to view the information for one of your previously looked up movies listed above
Enter 2 if you want to look up a new movie
Enter q to quit the program: 2

Please type in the movie title you would like to search for: avengers
We have found a match for The Avengers, here is the match for the most exact match or the match with the most recent year (2012)
The Avengers
Action, Sci-Fi
04 May 2012
143 min
Joss Whedon
Nominated for 1 Oscar. 38 wins & 80 nominations total

Loading more results matching your search query... 
H

---
## Section B: Design documentation

Here I will give an in depth explanation of my program, code logic and relevant design decisions.

### API explaination 

In order to understand how to interact with the OMDB RESTful API, let's have a look at firstly what a RESTful API is, and secondly how the OMDB API works.

#### What is a (RESTful) API?
An Application Program Interface (API) is a web-based interface and can be defined as a contract between applications that sets out the pattern of interactions between two application components. In layman's terms, it defines the rules on how to communicate with the program. API’s are tools that allow users to retrieve data from a database and use it. In other words, the API acts as an interface which the user can use to communicate with the program.

An API key is used to identify an entity (a user or program wishing to communicate with the API), and checks whether the entity has authorisation. If the entity has authorisation, then they can interact with the API according to the rules defined in the API.

An API consists primary of 3 main components:
1. the user (who makes requests)
2. the server (computers responding the requests from the user)
3. the client (computer that sends the request to a server)

A REpresentational State Transfer (RESTful) API is a software architectural style. In essense, when a client request is made through a RESTful API, it transfers a representation of the state of the resource on the server to the client. 
In order for an API to be considered RESTful, it must conform to six architectural constraints namely: uniform interface, stateless, catchable, client-server, layered system and code on demand.

#### How does the OMDB API work?

The Open Movie Database (OMDB) is a free website service which allows users to look up and search for infomation about movies from its database. A (private) API key needs to be requested to use the OMDB RESTful API. The user needs to use the API key in order to fetch and retrieve information about movies, and the API responds with the result in JSON format by default. To set your api key, one calls the `set_default()` function, with your API key as a parameter.

The two main functions used in this program is the omdb API's `get()` and `search()` functions.

The `get()` function retrieves generic response according to the keyword provided, and gives the response as a dictionary. At least the title or the imdbID needs to be provided. Further than that, additional parameters can be provided to customise the response, such as the year released, actors, director, etc. By default, if the title is provided with no additional parameters, then the movie/series/game matching that title the closest or has the most recent release date is returned.

The `search()` function retrieves a list of dictionaries containing the response. At least the title or the imdbID needs to be provided. Further than that, one can also provide the year and the type as optional parameters. The function retrieves a response in the format of a list of dictionaries with movies matching the imdbID or substring matches for the movie title provided.

There are also other more specialised functions in the omdb API, such as the `search_movie()`, `search_episode()`, `imdbid()` etc, but they were not used in this program as they are basically just subsets of the generic `get()` and `search()` functions.


Below showcases an example of how the `set_default()`, `get()`, and `search()` functions were used.


In [3]:
# setting the default API key
omdb.set_default('apikey', "48598600") 

# Generic get function by title, specifying a full response, 
# and allowing data from Rotten Tomato to be retrieved
print(omdb.get(title="Potter", fullplot=True, tomatoes=True))
print()
# Generic search by title (title set to "Potter for illustration purposes")
print(omdb.search("Potter"))

{'title': 'Potter', 'year': '1979–1983', 'rated': 'N/A', 'released': '01 Mar 1979', 'runtime': '30 min', 'genre': 'Comedy', 'director': 'N/A', 'writer': 'N/A', 'actors': 'Noel Dyson, John Barron, John Warner, Brenda Cowling', 'plot': 'Arthur Lowe as Redvers Potter owner of a mint manufacturers', 'language': 'English', 'country': 'UK', 'awards': 'N/A', 'poster': 'https://m.media-amazon.com/images/M/MV5BYzk1M2RhZDUtZDMzMS00NWE4LWI5MTEtNjAzYjhmNjQwYTY3XkEyXkFqcGdeQXVyMDY4MzkyNw@@._V1_SX300.jpg', 'ratings': [{'source': 'Internet Movie Database', 'value': '7.1/10'}], 'metascore': 'N/A', 'imdb_rating': '7.1', 'imdb_votes': '45', 'imdb_id': 'tt0078670', 'type': 'series', 'total_seasons': '3', 'tomato_meter': 'N/A', 'tomato_image': 'N/A', 'tomato_rating': 'N/A', 'tomato_reviews': 'N/A', 'tomato_fresh': 'N/A', 'tomato_rotten': 'N/A', 'tomato_consensus': 'N/A', 'tomato_user_meter': 'N/A', 'tomato_user_rating': 'N/A', 'tomato_user_reviews': 'N/A', 'tomato_url': 'N/A', 'dvd': 'N/A', 'box_office': 

### Code Explanation

Firstly my program introduces the service to the user then the program takes the user to the main menu and gives them the option of either viewing previous titles in their search , searching for a new movie title or quitting the program. The sample code out of my main program is displayed below showing this functionality.

#### Program Flow

##### Start of Program
At the start of the program 2 things happen: first, the user is introduced to the program, and second the titles and years of their previously searched movies is displayed if they have any search history.

The user is notified that they can quit the program or be directed to the main menu at any point by entering 'q' or 'm' respectively.

##### Main menu
The user gets 3 options:
1. Search for information about a movie that they have previously searched for
2. Search for a new movie
3. Quit the program

The program reacts according to the option that the user inputs. At most points throughout the program the user is given the option to return to this menu, or to quit the program.

##### Search for old movie
The user is presented with 2 options:
1. Search for information about a movie that they have previously searched for
2. See their search history (titles and years)

If they choose option 1, then they're requested the title and the year of the movie that they wish to retrieve informatation on. If they give an exact response that exists in their search history, then they are given the result. If they do not enter a valid response, then they are requested to re-enter their request.

If they choose option 2, then they can view their search history (if they have any). The program displays the titles and years of the movies that they have visited. They can use this information in their search query.

If the user types in 'q' or 'm' then they will quit the program or be directed to the main menu, respectively. If they quit the program, the user is thanked for using this service, and encouraged to use the program again.

##### Search for new movie
The user can search for information new movie in the omdb database.

The user is requested the title of the movie that they wish to retrieve informatation on. 
If their search query does not yield any response from the omdb database, then they are informed as such and have the option to search again. They are given a default response according to the most recent year of the title given or the most exact match to their input. In addition to this, they are given the years and titles of movies that also match their query. They are then given the option to get information of a more specific result by entering the year of that movie. 

If the user types in 'q' or 'm' then they will quit the program or be directed to the main menu, respectively. If they quit the program, the user is thanked for using this service, and encouraged to use the program again.

###### Writing to a file
If they are satisfied with the default response, or they provide an invalid year, then the default response is recorded in their search history by writing the result to a text file which can be loaded later (such as at the beginning of the program, and when searching for an old movie).

If they wish to get the result of a specific movie after getting the default response, then that result added to their search history by adding it to the text file as specified above.

### Noteworthy features: explaining key concepts

Note that the complete code is above in Section A. The code snippets below are only to display key functionalities and is not the final code.

##### Menu and quitting options

In each user input stage the user has the option of going back to the main menu (by pressing "m") or quitting the program (by pressing "q"). Here is a sample code to demonstrate the check. In the actual code, going back to the main menu would entail a break statement from the larger while loop (either in the new or the old search code), and in the case where the user quits, then the code returns "q", whereafter the main menu knows to exit the program.

See the sample code below to display functionality.

In [8]:
# sample code to showcase quitting and main menu option.
def check_menu_quit():
    while True:
        user_title = input("Give user input (checking for menu and quit cases): ")
        if user_title == "m" or user_title == "menu":
            break
        if user_title == "q" or user_title == "quit":
            return "q"
    return "Going to main menu"
print(check_menu_quit())

Give user input (checking for menu and quit cases): q
q


##### Search history

Firstly, the program checks if the Search History file exists or not. If the file does not exist, a new file called Search History is initialized and the user is informed that they have no previous search history. If the file does exist, the file is not initialised and at the beginning of the program  titles and years of the movies in the user’s search history is displayed. This is defined with the `get_history()` function as displayed in the below sample code. 


In [10]:
def get_history():
    # checks if the file exists, if it does not exist a new file is created 
    my_file = Path("search_history.txt")
    if not my_file.is_file():
        print("You have no search history, but let's work on that!")
        initialize_file()
        return
    
    # displays the movie title and years of the users previous search history, if they have one.
    search_array = []
    with open('search_history.txt', 'r') as f:
        search_array = json.loads(f.read())

    print("Here is your previous search history: ")

    for dict_history in search_array:
        print("Title:{} Year: {}".format(dict_history['title'], dict_history['released']))
        
get_history()

Here is your previous search history: 
Title:Harry Potter and the Deathly Hallows: Part 2 Year: 15 Jul 2011


##### Get previously searched movies

If the user chooses to search for a  movie they have previously searched for, then they should enter the title and the year of the movie. They can enter 'h' to see their previous search history (the search history is also displayed at the start of the program). To allow for a more user-friendly experience, the searching of the movie is not case sensitive and they only need to enter a substring of the title that they wish to retrieve to get the results (as long as the year is correct). 

If the user's input is valid, then the information about their queried movie is displayed. If they enter a response that does not result in a successful match in their search history, they are informed as such and allowed to search again. Their search is not outputted to the search history file. Note that this search function uses the information stored in the `search_history.txt` and does not make a new request to the database.

This is defined with the `search_old()` function, with a sample of it displayed below. 

In [ ]:
# gives user the option of searching for an old movie title, going back to the main menu or quitting the program
def search_old():
    while True:
        user_title = input("\nPlease type in the movie title you would like to search for.)
        user_year = input("\nPlease type in the movie year you would like to search for.)
        
        # checks if the movie title and year is found in the users search history
        search_array = []
        with open('search_history.txt', 'r') as f:
            search_array = json.loads(f.read())
        
        found_result = 0
        for dict_history in search_array:
            print(dict_history['released']) 
            if user_title.lower() in dict_history['title'].lower() and user_year == dict_history['released'][-4:]:
                print_user_output(dict_history['title'], dict_history['genre'], dict_history['released'], dict_history['runtime'], dict_history['director'], dict_history['awards'])
                found_result = 1
                break
        if found_result == 0:
            print("Search result does not match your previous search history. Please make sure that your movie title and year exactly matches your search history.")
        
search_old()

##### Search for new movies

To search for a new movie, the user needs to supply the title of the movie that they wish to search for. If they enter a title which does not yield a response from the omdb database, then they are displayed an appropriate message indicating that their search was not found. If their search is valid, they are given a default response according to omdb's `get()` function and given the option to get a more customised response if they provide a specific year. They are enabled to do this by the fact that the search matches (titles and years) are displayed underneath their default response, and a check is done beforehand to make sure that the title-year combination does yield a valid response. The years of the movies are sorted in descending order using Object Orientated Programming (discussed in the next section).

If they have not searched for the given movie before (i.e. it is not in their search history), then their search result is written to their history file. If they have searched for the movie before, then they are given a friendly message as a response and their result is not outputted to their search history. This is a design decision to maximise user experience when displaying the search history, such that duplicate searched aren't displayed and they can easily search for what they need.

In [15]:
user_title = "Harry Potter"

# the ombd get function searches the database for the title the user has provided
full_response = omdb.get(title=user_title, fullplot=True, tomatoes=True)

# if the title searched for was found, outputs the default response of information matching that title
if not full_response:
    print("\nYour search query was not found in our database, please search again")
print("We have found a match for {}, here is the match for the match for the most recent year ({})".format(full_response['title'], full_response['year']))
print_user_output(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])


# making a dictionary out of the information the user searched for 
default_response = make_summary_dict(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])

# using the search function to search through the database to find results matching the users search query
search_result = omdb.search(user_title) 

# displays additional movie titles also matching the search query below the default response 
print("Here are more results matching your search query: ")
search_result_entries = []
years_list = []
for movie_dict in search_result:
    if omdb.get(title=user_title, year=movie_dict['year']):
        entry = Entry(movie_dict['title'], movie_dict['year'])
        search_result_entries.append(entry)
        years_list.append(movie_dict['year'])

# Using OOP to sort entries by year
sorted_entries_results = sorted(search_result_entries, reverse=True)
for entry in sorted_entries_results:
    print("The title is: {}, The year is: {}".format(entry.title, entry.year))


We have found a match for Harry Potter and the Deathly Hallows: Part 2, here is the match for the match for the most recent year (2011)
Harry Potter and the Deathly Hallows: Part 2
Adventure, Drama, Fantasy, Mystery
15 Jul 2011
130 min
David Yates
Nominated for 3 Oscars. Another 46 wins & 91 nominations.

Here are more results matching your search query: 
The title is: Harry Potter and the Deathly Hallows: Part 2, The year is: 2011
The title is: Harry Potter and the Deathly Hallows: Part 1, The year is: 2010
The title is: Harry Potter and the Forbidden Journey, The year is: 2010
The title is: Harry Potter and the Half-Blood Prince, The year is: 2009
The title is: Harry Potter and the Order of the Phoenix, The year is: 2007
The title is: Harry Potter and the Goblet of Fire, The year is: 2005
The title is: Harry Potter and the Prisoner of Azkaban, The year is: 2004
The title is: Harry Potter and the Chamber of Secrets, The year is: 2002
The title is: Harry Potter and the Chamber of Secre

In [16]:
user_title = "Harry Potter"

# gives user the option to obtain more information about movie titles displayed below the default search results
user_year = input("\nIf you are interested in getting the results for movies with a different year with that title then enter one of the years outputted above, otherwise enter any key: ")
if user_year in years_list:
    full_response = omdb.get(title=user_title, year=user_year, fullplot=True, tomatoes=True)
    print_user_output(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])
    specific_response = make_summary_dict(full_response['title'], full_response['genre'], full_response['released'], full_response['runtime'], full_response['director'], full_response['awards'])

    # reads and writes file with data from the users specific search results or the users specific search results
    search_array = []
    with open('search_history.txt', 'r') as f:
        search_array = json.loads(f.read()) 
    if specific_response in search_array:
        print("Oeee! Searching for this movie again? Must be cool!\n")
    else:
        with open('search_history.txt', 'w') as file:
            search_array.append(specific_response)
            json.dump(search_array, file) 
# else write the default response in the previous file to the search history using similar code


If you are interested in getting the results for movies with a different year with that title then enter one of the years outputted above, otherwise enter any key: 2004
Harry Potter and the Prisoner of Azkaban
Adventure, Family, Fantasy, Mystery
04 Jun 2004
142 min
Alfonso Cuarón
Nominated for 2 Oscars. Another 17 wins & 51 nominations.



##### Object Orientated Programming (OOP)

I made use of OOP in my program as an additional feature to help me understand this cool concept and put my theory into practice. 

In my program I created a class with the name, `Entry()`. This class is used to store the title and year values retrieved by omdb's `search()` function, and allows for the entries to be sorted by year. This was done by initialising the object with the title and year, and overwriting the built-in greater than (`__gt__()`), greater than equals (`__gte__()`), and equals (`__eq__()`) Python methods. The `str()` and `repr()` functions can be used to represent the object in a human-understandable way.

In [ ]:
class Entry():
    # creates an object containing a number of methods and attributes
    def __init__(self, title, year):
        self.title = title
        self.year = year
    def __gt__(self, other_entry):
        return self.year > other_entry.year
    def __eq__(self, other_entry):
        return self.year == other_entry.year
    def __gte__(self, other_entry):
        return self.year >= other_entry.year
    def __str__(self):
        return f'Title: {self.title}, Year: {self.year}'
    def __repr__(self):
        return str(self)
    

In [19]:
user_title = "Harry Potter"
# using the search function to search through the database to find results matching the users search query
search_result = omdb.search(user_title)

# displays additional movie titles also matching the search query below the default response 
print("Here are more results matching your search query: ")
search_result_entries = []
years_list = []
for movie_dict in search_result:
    if omdb.get(title=user_title, year=movie_dict['year']):
        entry = Entry(movie_dict['title'], movie_dict['year'])
        search_result_entries.append(entry)
        years_list.append(movie_dict['year'])

# Using OOP to sort entries by year
sorted_entries_results = sorted(search_result_entries, reverse=True)
for entry in sorted_entries_results:
    print("The title is: {}, The year is: {}".format(entry.title, entry.year))

Here are more results matching your search query: 
The title is: Harry Potter and the Deathly Hallows: Part 2, The year is: 2011
The title is: Harry Potter and the Deathly Hallows: Part 1, The year is: 2010
The title is: Harry Potter and the Forbidden Journey, The year is: 2010
The title is: Harry Potter and the Half-Blood Prince, The year is: 2009
The title is: Harry Potter and the Order of the Phoenix, The year is: 2007
The title is: Harry Potter and the Goblet of Fire, The year is: 2005
The title is: Harry Potter and the Prisoner of Azkaban, The year is: 2004
The title is: Harry Potter and the Chamber of Secrets, The year is: 2002
The title is: Harry Potter and the Chamber of Secrets, The year is: 2002
The title is: Harry Potter and the Sorcerer's Stone, The year is: 2001


##### Formatting

I have included an additional feature that formats the output of the program using bold and different colours. The colours are customised according to the type of output (for example, green is a happy message and red is an error). This design was implemented to optimise user experience through increasing readability. 

In [36]:
# format text
bold = "\033[1m"
red = "\033[91m"
green = "\033[92m"
blue = "\033[94m"
end_formatting = "\033[0m"

print("{}This text is in bold{}".format(bold, end_formatting))
print("{}This text is in red{}".format(red, end_formatting))
print("{}This text is in green{}".format(green, end_formatting))
print("{}This text is in blue{}".format(blue, end_formatting))

This text is in bold
This text is in red
This text is in green
This text is in blue


### Retrospective

The purpose of the Retrospective is to provide a review and reflection of the overall project. In my Respospective, I clearly indentify and explain the areas in which I encountered challenges and I provide a description of what I did to solve to these issues. An evalution of potential improvements that could be made in the program is additionally identified and clearly explained along with what I have learned during this process.


#### Challenges and Solutions

*Understanding the API*
    
The first challenge I encountered in my project was that I did not clearly understand how a RESTful API worked. Additionally, I was confused about the process of how to interact with the OMDB RESTful API. My solution to this issue was that I spent many hours doing background research into finding out specifically what a RESTful API is and how to interact with the OMDB RESTful API, before starting my coding. I also went to tutorials to receive guidance on how I could tackle this project.
    
*Catering for invalid user input*

Another challenge I faced was figuring out how to handle unexpected inputs from a user. To solve this issue I ensured that my program was robust and basically tried breaking it to identify possible pitfalls. For these issues identified I subsequently wrote the code to handle these issues at every stage where the user is asked for input. I also considered a user-friendly design, allowing for the best user experience.
    
#### What I learned 

Through this project I learned how to successfully apply the theory covered in this course in practice. I found this to be challenging throughout the duration of the project as I had to do alot of research and use trial and error before I was able to get my program to sufficiently run. As a result of this project I have not only improved my understanding of the general theory covered in the course however I have gained additional experience in the  practical implementation of python coding.
    
#### What you would add

*Better user interface (GUI)*

Firstly, to improve my project I would have liked to have a better User interface. For example, the use of a Graphical User Interface. This could be implemented at the menu, for example, allowing the user to click the option they want instead of needing to type it in.

*More searching features*
    
It would have been ideal if users were able to search for actors, however this did not seem possible given that the get function requires a title (or an imdb ID, linking to a specific movie). Additionally, it would have been nice to show the poster of the movie that the user searches for.
 
  
*More Orientated Object Programming (OOP)*

A further improvement would be to include more Orientated Object Programming (OOP) in my main program. I would  like to have saved an individual user's search history and allowed users to create their own profiles to access and view their respective search histories.